In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
print("Current tf version is [%s]" % (tf.__version__))
print("Packages Loaded")

In [ ]:
mnist = input_data.read_data_sets('data/', one_hot=True)

In [ ]:
#NETWORK TOPOLOGIES
num_of_layer = 4 #전체 layer의 개수(input(1) + hidden 개수(2) + output(1))
num_of_nodes = [784, 256, 128, 10] #가장 처음 원소는 input의 node개수, 가장 마지막 원소는 output의 node 개수

#INPUT AND OUTPUTS
x = tf.placeholder("float", [None, num_of_nodes[0]]);
y = tf.placeholder("float", [None, num_of_nodes[len(num_of_nodes)-1]]);

#NETWORK PARAMETERS
stddev=0.1
weights=[]
biases=[]
for i in range(num_of_layer-1):
    weights.append(tf.Variable(tf.random_normal([num_of_nodes[i], num_of_nodes[i+1]], stddev=stddev)))
    biases.append(tf.Variable(tf.random_normal([num_of_nodes[i+1]])))
print("NETWORK READY")
    

In [ ]:
#MODEL
def multilayer_perceptron(_X, _weights, _biases):
    layers=[_X]
    for i in range(num_of_layer-1):
        layers.append(tf.nn.sigmoid(tf.add(tf.matmul(layers[i], _weights[i]), _biases[i])))
    return layers[len(layers)-1]

#PREDICTION
pred=multilayer_perceptron(x, weights, biases)

#LOSS AND OPTIMIZER
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
corr=tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accr = tf.reduce_mean(tf.cast(corr, "float"))

#INITIALIZER
init=tf.global_variables_initializer()
print("FUNCTIONS READY")

In [ ]:
# PARAMETERS
training_epochs = 20
batch_size = 100
display_step = 4
# LAUNCH THE GRAPH
sess = tf.Session()
sess.run(init)
#OPTIMIZE
for epoch in range(training_epochs):
    avg_cost = 0.
    total_batch = int(mnist.train.num_examples/batch_size)
    # ITERATION
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        feeds = {x: batch_xs, y:batch_ys}
        sess.run(optm, feed_dict=feeds)
        avg_cost += sess.run(cost, feed_dict=feeds)
    avg_cost = avg_cost / total_batch
    # DISPLAY
    if (epoch+1) % display_step == 0:
        print ("Epoch : %03d/%03d cost : %.9f" % (epoch+1, training_epochs, avg_cost))
        feeds = {x: batch_xs, y: batch_ys}
        train_acc = sess.run(accr, feed_dict=feeds)
        print ("TRAIN ACCURACY : %.3f" % (train_acc))
        feeds = {x: testimgs, y: testlabels}
        test_acc = sess.run(accr, feed_dict=feeds)
        print ("TEST ACCURACY: %.3f" % (test_acc))
        
print ("OPTIMZATION FINISHED")
